# To create dynamic directories based on PDF name
When running these series of commands you will be able to filter the pdfs based on their name

In [101]:
# To extract the tables from the PDFS in the iteration, pass down the START_MARKERS and END_MARKERS as props
import pandas as pd

def ExtractTable(data_array, START_MARKERS, END_MARKERS):
    for text in data_array:
        extracted_text = []

        for start_marker in START_MARKERS:
            start_index = text.find(start_marker) 

            if start_index != -1:
                for end_marker in END_MARKERS:
                    end_index = text.find(end_marker, start_index)
                    if end_index != -1:
                        extracted_text.append(text[start_index:end_index])

    return extracted_text

def FlattenList(nested_list):
    flat_list = []
    
    for item in nested_list:
        flat_list += item

    return flat_list

def CreateDataFrame(str_table):
    count = 0
    for line in str_table:
        count += 1
        lines = line.strip().split('\n')
        header = lines[2].split()
        data = [line.split() for line in lines[5:]]
        
    return pd.DataFrame(data, columns = header)


In [109]:
import os
import re
import PyPDF2

DIRECTORY_PATH = '../pdfs/'

pdf_files=[file for file in os.listdir(DIRECTORY_PATH) if file.endswith('.pdf')]

dataframe = pd.DataFrame(columns=['Peak', 'RetTime', 'Type', 'Width', 'Area', 'Height', 'Area', 'Sample'])

for file_name in pdf_files:
    match = re.search(r'(.+ \d{4}-\d{2}-\d{2}|\d{2}-\d{2}-\d{2}) (.+?)\.pdf', file_name)
    if match:
        common_part = match.group(2)
        if 'Mel' in common_part or 'HM' in common_part:
            with open(DIRECTORY_PATH + file_name, 'rb') as pdf_file:
                pdf = PyPDF2.PdfReader(pdf_file)
                pdf_pages = [pdf.pages[i].extract_text() for i in range(1, len(pdf.pages))]
                str_table = ExtractTable(pdf_pages, ['Signal 1:'], ['Totals :'])
                df = CreateDataFrame(str_table)

                row_number = 1 if 'Mel' in common_part and len(df) >= 2 else (2 if len(df) > 2 else 1)
                row_to_add = df.iloc[row_number, :].copy()
                
                common_part = common_part.replace(' X2', '').replace(' ', '')
                row_to_add['Sample'] = common_part

                dataframe = pd.concat([dataframe, row_to_add.to_frame().T], ignore_index=True)


print(dataframe)




   Peak RetTime Type   Width        Area      Height     Area     Sample
0     3   2.724   BB  0.0676     8.95197     2.02566  15.9640   HM0.1ppm
1     3   2.730   BB  0.0701     8.97839     2.01094  16.4159   HM0.1ppm
2     3   1.534   VB  0.0525    52.03939    14.99019  56.5095   HM0.5ppm
3     3   1.535   VB  0.0548    53.55626    15.31030  57.2943   HM0.5ppm
4     3   1.494   VB  0.1024   797.48700   110.29391  93.5271    HM10ppm
5     3   1.504   VB  0.0937   888.22400   144.42703  93.9590    HM10ppm
6     3   1.535   VB  0.0553   102.35654    28.93162  59.7338     HM1ppm
7     3   1.536   VB  0.0555   105.41412    29.61987  60.3783     HM1ppm
8     2   1.501   VV  0.0945  1385.79517   217.03145  96.6831    HM20ppm
9     3   1.495   VV  0.1015  1363.35608   195.20973  95.3379    HM20ppm
10    3   1.732   VV  0.0670    40.36325     8.88480   0.8956    HM50ppm
11    3   1.496   VV  0.1168  4019.03174   566.19232  96.4909    HM50ppm
12    3   1.535   VV  0.0552   604.27716   170.9723